# Scalar Value Encoding

**Topics:** FractionalPowerEncoder, ThermometerEncoder, LevelEncoder
**Time:** 20 minutes
**Prerequisites:** 00_quickstart.ipynb, 01_basic_operations.ipynb

---

## Setup

In [ ]:
from holovec import VSA
from holovec.encoders import FractionalPowerEncoder, ThermometerEncoder, LevelEncoder
import numpy as np

model = VSA.create('FHRR', dim=10000, seed=42)
print(f"Model: {model.model_name}")

## FractionalPowerEncoder

The **FractionalPowerEncoder** uses complex exponentials to encode continuous values with smooth similarity.

In [ ]:
# Create encoder for temperature (0-100°C)
fpe = FractionalPowerEncoder(model, min_val=0, max_val=100, bandwidth=0.1, seed=42)

print(f"Encoder: {fpe}")
print(f"Reversible: {fpe.is_reversible}")

In [ ]:
# Encode temperatures
temps = [20.0, 21.0, 25.0, 50.0]
encoded = {t: fpe.encode(t) for t in temps}

print("\nSimilarity matrix:")
for t1 in temps:
    sims = [float(model.similarity(encoded[t1], encoded[t2])) for t2 in temps]
    print(f"{t1:5.1f}°C: " + "  ".join(f"{s:.3f}" for s in sims))

### Decoding

FPE is reversible - we can decode hypervectors back to approximate scalar values.

In [ ]:
# Decode a hypervector
test_temp = 37.5
test_hv = fpe.encode(test_temp)
decoded = fpe.decode(test_hv)

print(f"Original: {test_temp}°C")
print(f"Decoded:  {decoded:.2f}°C")
print(f"Error:    {abs(test_temp - decoded):.2f}°C")

## ThermometerEncoder

**ThermometerEncoder** uses ordinal encoding - active bins from min to current value.

In [ ]:
# Create thermometer encoder
thermo = ThermometerEncoder(model, min_val=0, max_val=100, n_bins=20)

print(f"Encoder: {thermo}")
print(f"Bins: {thermo.n_bins}")
print(f"Reversible: {thermo.is_reversible}")

In [ ]:
# Encode and compare
temps_test = [10.0, 15.0, 80.0]
thermo_encoded = {t: thermo.encode(t) for t in temps_test}

print("\nThermometer encoding similarities:")
for t1 in temps_test:
    for t2 in temps_test:
        if t1 < t2:
            sim = float(model.similarity(thermo_encoded[t1], thermo_encoded[t2]))
            print(f"  {t1}°C vs {t2}°C: {sim:.3f}")

## LevelEncoder

**LevelEncoder** creates discrete bins for value ranges.

In [ ]:
# Create level encoder
level = LevelEncoder(model, min_val=0, max_val=100, n_levels=5)

print(f"Encoder: {level}")
print(f"Levels: {level.n_levels}")
print(f"Reversible: {level.is_reversible}")

In [ ]:
# Encode into discrete levels
test_values = [5.0, 15.0, 45.0, 75.0, 95.0]
level_encoded = {v: level.encode(v) for v in test_values}

print("\nLevel encoding:")
for v in test_values:
    decoded = level.decode(level_encoded[v])
    print(f"  {v:5.1f} → level bin → {decoded:5.1f}")

## Encoder Comparison

Let's compare all three encoders on the same data.

In [ ]:
test_val = 42.0

fpe_hv = fpe.encode(test_val)
thermo_hv = thermo.encode(test_val)
level_hv = level.encode(test_val)

print(f"Encoding {test_val}°C:\n")
print(f"FPE:         Reversible={fpe.is_reversible}, Decoded={fpe.decode(fpe_hv):.2f}°C")
print(f"Thermometer: Reversible={thermo.is_reversible}")
print(f"Level:       Reversible={level.is_reversible}, Decoded={level.decode(level_hv):.2f}°C")

## Summary

✓ **FractionalPowerEncoder**: Smooth similarity, reversible, best for continuous values  
✓ **ThermometerEncoder**: Ordinal encoding, fast, good for rankings  
✓ **LevelEncoder**: Discrete bins, reversible, good for categorical ranges

### When to use each:
- **FPE**: Temperature, pressure, continuous measurements
- **Thermometer**: Rankings, scores, ordinal data
- **Level**: Binned data, categorical ranges

### Next Steps
- `14_encoders_ngram.ipynb` - Text encoding
- `16_encoders_vector.py` - Multivariate encoding